In [1]:
import pandas as pd

In [2]:
juegos = pd.read_csv('archivos_csv/games_final.csv')
items = pd.read_csv('archivos_csv/items_final_pulido.csv')
reviews = pd.read_csv('archivos_csv/reviews_sentiment_analysis.csv')

In [3]:
juegos.head()

,app_name,release_date,tags,id
0,Lost Summoner Kitty,2018.0,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140.0
1,Ironbound,2018.0,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",643980.0
2,Real Pool 3D - Poolians,2017.0,"['Free to Play', 'Simulation', 'Sports', 'Casu...",670290.0
3,弹炸人2222,2017.0,"['Action', 'Adventure', 'Casual']",767400.0
4,Log Challenge,NaN,"['Action', 'Indie', 'Casual', 'Sports']",773570.0


In [4]:
reviews.head()

,user_id,funny,posted,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,NaN,2011,1250,No ratings yet,True,1
1,76561197970982479,NaN,2011,22200,No ratings yet,True,1
2,76561197970982479,NaN,2011,43110,No ratings yet,True,1
3,js41637,NaN,2014,251610,15 of 20 people (75%) found this review helpful,True,1
4,js41637,NaN,2013,227300,0 of 1 people (0%) found this review helpful,True,1


In [153]:
items.head(1)

,user_id,item_id,item_name,playtime_forever
0,phrostb,300,Day of Defeat: Source,78


In [9]:
#armo df acorde a las necesidades de la función
reviews_fecha = reviews.drop(columns=['funny', 'helpful', 'recommend', 'sentiment_analysis']) #elimino las columnas que no me sirven
juegos_años = pd.merge(items, reviews_fecha, on= ['user_id', 'item_id'], how='inner') #concateno para tener items y fecha de review en un mismo df

In [10]:
juegos_años.head()

,user_id,item_id,item_name,playtime_forever,posted
0,phrostb,70,Half-Life,21.716667,2014
1,phrostb,620,Portal 2,22.833333,2014
2,phrostb,45760,Ultra Street Fighter IV,29.200000,2013
3,phrostb,238210,System Shock 2,26.233333,2014
4,phrostb,207750,Symphony,0.700000,2014


In [78]:
juegos_años.posted.unique()

array([2014, 2013, 2015, 2010, 2012, 2011])

In [101]:
def UserForGenre(genero):
    #obtengo liste de ids de juegos que se corresponden con ese género
    juegos_id = juegos[juegos.tags.apply(lambda x:(genero.capitalize() in x) )]['id']
    #creo diccionario con jugadores y horas jugadas
    jugadores = juegos_años.user_id.unique()
    horas_jugadas = { jugador:0 for jugador in jugadores}
   #filtro el df con los ids de los juegos que responden al género 
    juegos_filtrados = juegos_años[juegos_años['item_id'].isin(juegos_id)]
    # itero para sumar horas jugadas para cada jugador
    for indice, fila in juegos_filtrados.iterrows():
        horas_jugadas[fila['user_id']] += fila['playtime_forever']
    #jugador con más horas jugadas
    max_user = max(horas_jugadas, key=horas_jugadas.get)
    # diccionario para calcular años con más horas jugadas.
    horas_por_año = {año:0 for año in juegos_años.posted.unique()}

    juegos_filtrados_por_usuario = juegos_años[(juegos_años['item_id'].isin(juegos_id)) & (juegos_años['user_id'] == max_user)]

    for indice, fila in juegos_filtrados_por_usuario.iterrows():
        horas_por_año[fila['posted']] += fila['playtime_forever']

    
    horas_por_año_ordenado = sorted(horas_por_año.items(), key=lambda x: x[1], reverse=True)
    
    horas_jugadas_top = [{"Año": año, "Horas": horas} for año, horas in horas_por_año_ordenado[:3] if horas > 0]
    
    return (f"Usuario con más horas jugadas para Género {genero.capitalize()}: {max_user}, Horas jugadas: {horas_jugadas_top}")

In [102]:
UserForGenre('ACTION')

"Usuario con más horas jugadas para Género Action: shinomegami, Horas jugadas: [{'Año': 2015, 'Horas': 594498}, {'Año': 2012, 'Horas': 137029}, {'Año': 2014, 'Horas': 8392}]"

In [103]:
def PlayTimeGenre(genero):
    #obtengo lista de ids de juegos que se corresponden con ese género
    juegos_id = juegos[juegos.tags.apply(lambda x:(genero.capitalize() in x) )]['id']
    #filtro el df con los ids de los juegos que responden al género 
    juegos_filtrados = juegos_años[juegos_años['item_id'].isin(juegos_id)]
    # itero para sumar horas jugadas para cada jugador
    horas_por_release_year = {year:0 for year in juegos.release_date.unique()}
    for indice, fila in juegos_filtrados.iterrows():
        filtro = juegos['id'] == fila['item_id']
        if isinstance(juegos.loc[filtro, 'release_date'], int): ## este filtro es para eliminar los juegos con release_date nulo
            año_lanzamiento = int(juegos.loc[filtro, 'release_date'])
            horas_por_release_year[año_lanzamiento]+= fila['playtime_forever']
    
    año = max(horas_por_release_year, key=horas_por_release_year.get)

    return f"Año de lanzamiento con más horas jugadas para Género {genero.capitalize()} : {año}"
    
    

In [104]:
PlayTimeGenre('CaSuAl')

'Año de lanzamiento con más horas jugadas para Género Casual : 2018.0'

In [166]:
def UsersRecommend(año):
    if año not in reviews.posted.unique():
        return f"No hay recomendaciones para ese año. Pruebe con uno de estos años: {reviews.posted.unique()}"
    
    df = reviews[(reviews.posted == año)&(reviews['item_id'].isin(juegos.id))][['posted', 'item_id', 'recommend', 'sentiment_analysis']]

    juegos_recomendados = {item :0 for item in df.item_id.unique()}
    
    for indice, fila in df.iterrows():
        if fila.recommend is True:
            juegos_recomendados[fila['item_id']] +=1
        juegos_recomendados[fila['item_id']] += fila.sentiment_analysis
    
    juegos_recomendados = sorted(juegos_recomendados.items(), key=lambda x: x[1], reverse=True)
    
    juego1 = str(juegos[juegos['id'] == juegos_recomendados[0][0]]['app_name'].iloc[0])
    juego2 = str(juegos[juegos['id'] == juegos_recomendados[1][0]]['app_name'].iloc[0])
    juego3 = str(juegos[juegos['id'] == juegos_recomendados[2][0]]['app_name'].iloc[0])

    return f"Puesto 1 : {juego1}, Puesto 2 : {juego2}, Puesto 3 : {juego3}"
    

In [157]:
UsersRecommend(2015)

"Puesto 1 : Counter-Strike: Global Offensive, Puesto 2 : Team Fortress 2, Puesto 3 : Garry's Mod"

In [158]:
def UsersNotRecommend(año):
    if año not in reviews.posted.unique():
        return f"No hay recomendaciones para ese año. Pruebe con uno de estos años: {reviews.posted.unique()}"
    
    df = reviews[(reviews.posted == año)&(reviews['item_id'].isin(juegos.id))][['posted', 'item_id', 'recommend', 'sentiment_analysis']]

    juegos_recomendados = {item :0 for item in df.item_id.unique()}
    
    for indice, fila in df.iterrows():
        if fila.recommend is False:
            juegos_recomendados[fila['item_id']] +=1
        if fila.sentiment_analysis == 0:
            juegos_recomendados[fila['item_id']] +=1
    
    juegos_recomendados = sorted(juegos_recomendados.items(), key=lambda x: x[1], reverse=True)
    
    juego1 = str(juegos[juegos['id'] == juegos_recomendados[0][0]]['app_name'].iloc[0])
    juego2 = str(juegos[juegos['id'] == juegos_recomendados[1][0]]['app_name'].iloc[0])
    juego3 = str(juegos[juegos['id'] == juegos_recomendados[2][0]]['app_name'].iloc[0])

    return f"Puesto 1 : {juego1}, Puesto 2 : {juego2} , Puesto 3 : {juego3}"

In [167]:
UsersNotRecommend(2015)

'Puesto 1 : Counter-Strike: Global Offensive, Puesto 2 : PAYDAY 2 , Puesto 3 : DayZ'

In [164]:
def SentimentAnalysis(año):
    juegos_de_ese_año = list(juegos[juegos['release_date']== año].id)
    reseñas = list(reviews[reviews['item_id'].isin(juegos_de_ese_año)]['sentiment_analysis'])
    if len(reseñas) == 0:
        return "No hay reseñas para juegos lanzados ese año"
    else:
        return f"Negative = {reseñas.count(0)}, Neutral = {reseñas.count(1)}, Positive = {reseñas.count(2)}"
    

In [127]:
juegos.release_date.unique()

array([2018., 2017.,   nan, 1997., 1998., 2016., 2006., 2005., 2003.,
       2007., 2002., 2000., 1995., 1996., 1994., 2001., 1993., 2004.,
       1999., 2008., 2009., 1992., 1989., 2010., 2011., 2013., 2012.,
       2014., 1983., 1984., 2015., 1990., 1988., 1991., 1985., 1982.,
       1987., 1981., 1986., 2021., 2019., 1975., 1970., 1980.])

In [165]:
SentimentAnalysis(2003)

'Negative = 5, Neutral = 178, Positive = 3'

In [123]:
list(reviews.sentiment_analysis).count(2)

4513

In [109]:
juegos.head(1)

,app_name,release_date,tags,id
0,Lost Summoner Kitty,2018.0,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140.0
